In [1]:
from soma import aims
import pandas as pd
import numpy as np
from pathlib import Path

from typing import Dict

# Paths for the analysis

<span style="color:red"> Analysis on V1 subject in ataxia, and on PN (it's a preprocessing done on the data -> cf. Clara)</span>

In [2]:
class PathSubjectAtaxia : 
    def __init__(self, subject_id): 
        # Paths template
        # DB location
        ATAXIA_ZSUN = Path("/neurospin/dico/zsun/ataxie/etudes_AlexandraDurr/database_brainvisa/cermoi")
        CATI_CERMOI_DB = Path("/neurospin/cati/cati_members/studies_cati/cermoi/database_brainvisa/00")

        # Tree in the DB to access the graphs 
        # Ataxia zsun
        TREE_GRAPH = Path("t1mri/V1/default_analysis/folds/3.1")
        # Cati processed
        TREE_CERES = Path("cereb_bs4/V1")
        NOMENCLATURE_CEREBELLUM_SEG = "_cerebellum_brainstem_split_mask.nii.gz"

        # subject specific paths
        self.id = subject_id
        self.graph = ATAXIA_ZSUN / subject_id / TREE_GRAPH / f"R{subject_id}.arg"
        self.cerebellum_mask = CATI_CERMOI_DB / subject_id / TREE_CERES / f"{subject_id}{NOMENCLATURE_CEREBELLUM_SEG}"

In [3]:
SUBJECT = "00001PJ"
sub_path = PathSubjectAtaxia(SUBJECT)
sub_path.graph, sub_path.cerebellum_mask

(PosixPath('/neurospin/dico/zsun/ataxie/etudes_AlexandraDurr/database_brainvisa/cermoi/00001PJ/t1mri/V1/default_analysis/folds/3.1/R00001PJ.arg'),
 PosixPath('/neurospin/cati/cati_members/studies_cati/cermoi/database_brainvisa/00/00001PJ/cereb_bs4/V1/00001PJ_cerebellum_brainstem_split_mask.nii.gz'))

# Retrieving cerebllum mask from CERES

We are working with the segmentation that has been done with by Clara Fischer in the ataxia project.

The segmentation has been done using Ceres for the lobes of the brain and in addition to that, the vermis have segmented using vol2brain, those 2 segmentation have been merged to have those parts : 
- Two hemispheres (separated components)
- Vermis 
- Brain stem structure (3 different components)

Only the 2 cerebellum hemisphere and the vermis are interesting for us.

### Cerebellum mask

In [4]:
aims_obj = aims.read(str(sub_path.cerebellum_mask))
vol_np = aims_obj.np
np.unique(vol_np)

array([0, 1, 2, 3, 5, 6, 7, 8], dtype=int16)

Detail of the encoding of the different parts : 
- `0` : Background
- `1` : Hemisphere
- `2` : Hemisphere
- `3` : Vermis 
- `[5-8]` : Brainstem

In [5]:
# Changing the values of the volume
def only_mask_cerebellum(vol : np.ndarray) -> None :
    cerebellum = np.isin(vol_np, [1,2,3])
    rest = np.isin(vol_np, [0,5,6,7,8])

    vol_np[cerebellum] = 1
    vol_np[rest] = 0

In [6]:
only_mask_cerebellum(vol_np)
np.unique(vol_np)

array([0, 1], dtype=int16)

In [7]:
SAVING_FOLDER = Path("/neurospin/tmp/tsanchez/cerebellum_mask/ataxia")
# aims.write(obj = aims_obj, filename= str(SAVING_FOLDER / f"{SUBJECT}_cereb_bs4_cerebellum_only.nii.gz"))

### Retreiving transform matrix to ICBM2009c w/ `deep_folding`

We need to transform to register the mask we obtained to IBCM2009c

In [8]:
def get_ICBM2009c_transform(graph_path) : 
    graph = aims.read(str(graph_path))
    transf = aims.GraphManip.getICBM2009cTemplateTransform(graph)
    return transf

In [9]:
mask_transform = get_ICBM2009c_transform(sub_path.graph)

In [10]:
from deep_folding.brainvisa.utils.resample import resample

In [11]:
RESAMPLE_VALUES = [1]
OUTPUT_VOXEL_SIZE = (1,1,1)
resampled_cereb_mask = resample(
    input_image=aims_obj, 
    transformation=mask_transform,
    output_vs=OUTPUT_VOXEL_SIZE,
    background=0,
    values= RESAMPLE_VALUES,
    verbose=True
)

DEBUG:resample.py: Time before resampling: 0.12787437438964844s
DEBUG:resample.py: Background resampling: 0.01322031021118164s
DEBUG:resample.py: Time: 0.7795796394348145s
DEBUG:resample.py: 	0.19052863121032715s to create the bucket
	0.30269503593444824s to resample bucket
	0.03833889961242676s to assign values


restoring values


In [12]:
import deep_folding.brainvisa.utils.dilate_mask as dl 

DILATATION = 5 #mm
dilated_mask = dl.dilate(resampled_cereb_mask, radius=DILATATION)
aims.write(dilated_mask, filename = str(SAVING_FOLDER / f"{SUBJECT}_ICBM2009c_cerebellum_mask_{DILATATION}mm_dilatation.nii.gz"))

### Register pclean and hcp subjects to ICBM2009c

In [13]:
class PathSubject:
    def __init__(self, subject_id : str, db : str):
        assert db in ["hcp", "pclean"]

        PATH_DATA = Path("/neurospin/dico/data/bv_databases/human")
        PATH_LOCAL = Path(f"/neurospin/tmp/tsanchez/{db}")


        if db == "pclean" :
            PATH_DB = PATH_DATA / "manually_labeled" / "pclean" / "all" 
            TREE_GRAPH = Path("t1mri/t1/default_analysis/folds/3.1")
        else :
            PATH_DB = PATH_DATA / "automatically_labeled" / "hcp"/ "hcp"
            TREE_GRAPH = Path("t1mri/BL/default_analysis/folds/3.1")

        self.id = subject_id
        self.local = PATH_LOCAL
        self.graph = PATH_DB / subject_id / TREE_GRAPH / f"R{subject_id}.arg"
        self.thresh = PATH_LOCAL / "thresh" / f"threshold_cerebellum_{subject_id}.nii.gz"
        self.ICBM2009c = PATH_LOCAL / "ICBM2009c_thresh" / f"{subject_id}_ICBM2009c_thresh.nii.gz"
        self.final = PATH_LOCAL / "cerebellum_thresholded" / f"{subject_id}_cerebellum_only_thresh.nii.gz"
        self.white_matter = PATH_LOCAL / "cerebellum_thresholded" / f"{subject_id}_white_matter.nii.gz"
        self.sulci = PATH_LOCAL / "cerebellum_thresholded" / f"{subject_id}_sulci.nii.gz"
    
    def __repr__(self) : 
        return f"Paths({self.id})"

In [14]:
test = 518746
path_test = PathSubject(str(test), db = "hcp")
path_test.id, path_test.local, path_test.graph, path_test.thresh, path_test.ICBM2009c, path_test.final


('518746',
 PosixPath('/neurospin/tmp/tsanchez/hcp'),
 PosixPath('/neurospin/dico/data/bv_databases/human/automatically_labeled/hcp/hcp/518746/t1mri/BL/default_analysis/folds/3.1/R518746.arg'),
 PosixPath('/neurospin/tmp/tsanchez/hcp/thresh/threshold_cerebellum_518746.nii.gz'),
 PosixPath('/neurospin/tmp/tsanchez/hcp/ICBM2009c_thresh/518746_ICBM2009c_thresh.nii.gz'),
 PosixPath('/neurospin/tmp/tsanchez/hcp/cerebellum_thresholded/518746_cerebellum_only_thresh.nii.gz'))

In [15]:
### Subjects 
dict_subjects = {
                "hcp" : [
                        "163331", 
                        "518746", 
                        "991267", 
                        ],
                "pclean" : [
                        "s12158", 
                        "s12401", 
                        "s12635", 
                        ]
                }

dict_path = {
    "hcp" : [PathSubject(sub_id, "hcp") for sub_id in dict_subjects["hcp"]],
    "pclean" : [PathSubject(sub_id, "pclean") for sub_id in dict_subjects["pclean"]],
}

dict_path

{'hcp': [Paths(163331), Paths(518746), Paths(991267)],
 'pclean': [Paths(s12158), Paths(s12401), Paths(s12635)]}

In [18]:
def transform_ICBM2009c(paths : PathSubject, save : bool = False) : 

    # TODO : Add this as arguments
    RESAMPLE_VALUES = [0, -1, 1]
    OUTPUT_VOXEL_SIZE = (1,1,1)

    # !!! Not really the native object here
    native_obj = aims.read(str(paths.thresh))
    c = aims.Converter(intype=native_obj, outtype=aims.Volume('S16'))
    native_obj = c(native_obj)

    # Apply transform
    transf = get_ICBM2009c_transform(paths.graph)
    resampled_to_ICBM2009c = resample(
        input_image=native_obj, 
        transformation=transf,
        output_vs=OUTPUT_VOXEL_SIZE,
        background=0,
        values= RESAMPLE_VALUES,
        verbose=True
    )

    if save : 
        aims.write(resampled_to_ICBM2009c, filename=str(paths.ICBM2009c))

    return resampled_to_ICBM2009c


In [19]:
dict_registered_ICBMc2009 = {
    "hcp" : [transform_ICBM2009c(path_sub, save=True) for path_sub in dict_path["hcp"]],
    "pclean" : [transform_ICBM2009c(path_sub, save=True) for path_sub in dict_path["pclean"]],
}

DEBUG:resample.py: Time before resampling: 0.027959823608398438s
DEBUG:resample.py: Background resampling: 0.004275321960449219s
DEBUG:resample.py: Time: 16.01353168487549s
DEBUG:resample.py: 	5.679744482040405s to create the bucket
	9.257164478302002s to resample bucket
	0.5303919315338135s to assign values


restoring values


DEBUG:resample.py: Time before resampling: 0.01725602149963379s
DEBUG:resample.py: Background resampling: 0.007330417633056641s
DEBUG:resample.py: Time: 14.994313478469849s
DEBUG:resample.py: 	5.458379507064819s to create the bucket
	8.518330335617065s to resample bucket
	0.48970842361450195s to assign values


restoring values


DEBUG:resample.py: Time before resampling: 0.01785731315612793s
DEBUG:resample.py: Background resampling: 0.005156993865966797s
DEBUG:resample.py: Time: 13.820917844772339s
DEBUG:resample.py: 	5.182499647140503s to create the bucket
	7.669177532196045s to resample bucket
	0.44927072525024414s to assign values


...

restoring values


DEBUG:resample.py: Time before resampling: 0.008360862731933594s
DEBUG:resample.py: Background resampling: 0.0048902034759521484s
DEBUG:resample.py: Time: 6.896492958068848s
DEBUG:resample.py: 	1.4144809246063232s to create the bucket
	5.034775018692017s to resample bucket
	0.22500848770141602s to assign values


restoring values


DEBUG:resample.py: Time before resampling: 0.008545875549316406s
DEBUG:resample.py: Background resampling: 0.0051822662353515625s
DEBUG:resample.py: Time: 7.577647686004639s
DEBUG:resample.py: 	1.4006574153900146s to create the bucket
	5.6677632331848145s to resample bucket
	0.287036657333374s to assign values


restoring values


### Applying Mask to ICBM2009c Threshold

In [20]:
# Mask file path
mask_file = SAVING_FOLDER / f"{SUBJECT}_ICBM2009c_cerebellum_mask_{DILATATION}mm_dilatation.nii.gz"
mask_file

PosixPath('/neurospin/tmp/tsanchez/cerebellum_mask/ataxia/00001PJ_ICBM2009c_cerebellum_mask_5mm_dilatation.nii.gz')

In [21]:
# Readgin mask file
mask = aims.read(str(mask_file))
mask_np = mask.np
np.unique(mask_np)

array([0, 1], dtype=int16)

In [22]:
# Selecting hcp subject
subject_hcp = dict_path["hcp"][0]
subject_pclean = dict_path["pclean"][0]
subject_hcp, subject_pclean

(Paths(163331), Paths(s12158))

In [23]:
# Settings mask to bool 
mask_bool = np.where(mask_np == 1, True, False)
np.unique(mask_bool)

array([False,  True])

In [24]:
# Reding files
vol_hcp, vol_pclean = aims.read(str(subject_hcp.ICBM2009c)), aims.read(str(subject_pclean.ICBM2009c)) 
vol_hcp_np, vol_pclean_np = vol_hcp.np, vol_pclean.np 

In [25]:
# Applying cerebellum mask
vol_hcp_np[~mask_bool] = 0
vol_pclean_np[~mask_bool] = 0

In [26]:
# Saving files 
aims.write(vol_hcp, str(subject_hcp.final))
aims.write(vol_pclean, str(subject_pclean.final))

### Select only white matter and sulci


In [36]:
# Loading files
hcp_sulci , hcp_white_matter = aims.read(str(subject_hcp.ICBM2009c)), aims.read(str(subject_hcp.ICBM2009c))
pclean_sulci , pclean_white_matter = aims.read(str(subject_pclean.ICBM2009c)), aims.read(str(subject_pclean.ICBM2009c))

In [37]:
# Retrieving np volumes
hcp_sulci_np = hcp_sulci.np 
hcp_white_matter_np = hcp_white_matter.np 
pclean_sulci_np = pclean_sulci.np 
pclean_white_matter_np = pclean_white_matter.np 

In [40]:
# TODO : check why it need to run 2 times ? 
# Retrieving only white matter
hcp_white_matter_np[:] = np.where(hcp_white_matter_np == -1, 1,0)
pclean_white_matter_np[:] = np.where(pclean_white_matter_np == -1, 1,0)

# Retrieving only sulci
hcp_sulci_np[:] = np.where(hcp_sulci_np == 1, 1,0)
pclean_sulci_np[:] = np.where(pclean_sulci_np == 1, 1,0)


# Retrieving only white matter
hcp_white_matter_np[~mask_bool] = 0
pclean_white_matter_np[~mask_bool] = 0

# Retrieving only sulci
hcp_sulci_np[~mask_bool] = 0
pclean_sulci_np[~mask_bool] = 0


In [41]:
# Need to be [0,1] every where
np.unique(hcp_sulci_np), np.unique(hcp_white_matter_np), np.unique(pclean_white_matter_np), np.unique(pclean_sulci_np)

(array([0, 1], dtype=int16),
 array([0, 1], dtype=int16),
 array([0, 1], dtype=int16),
 array([0, 1], dtype=int16))

In [43]:
hcp_sulci

In [42]:
aims.write(hcp_sulci, filename = str(subject_hcp.sulci))
aims.write(hcp_white_matter, filename = str(subject_hcp.white_matter))
aims.write(pclean_sulci, filename = str(subject_pclean.sulci))
aims.write(pclean_white_matter, filename = str(subject_pclean.white_matter))